In [1]:
import numpy as np
import os
import sys

from tensorflow import keras

from keras.models import Sequential, Model, load_model
from keras.layers.core import Dense, Activation
from keras.layers import Permute, multiply, Dot, LSTM, Input, Flatten, Concatenate, Embedding, Convolution1D,Dropout, Conv2D, Conv1D, Bidirectional, MaxPooling1D, Reshape
from keras.layers.wrappers import TimeDistributed

from keras.optimizers import SGD, Adam, RMSprop
from keras.layers.normalization import BatchNormalization
from sklearn.preprocessing import label_binarize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.regularizers import l2

import librosa
import argparse
import keras
import tensorflow as tf
from keras import backend as K
from keras import regularizers, constraints, initializers, activations
from keras.layers.recurrent import Recurrent
from keras.engine import InputSpec
from keras.callbacks import EarlyStopping,TensorBoard, ModelCheckpoint
from keras_self_attention import SeqSelfAttention
from keras_multi_head import MultiHeadAttention

from spektral.layers import GraphConv, GraphAttention
from spektral.layers.ops import sp_matrix_to_sp_tensor
from spektral.utils import normalized_laplacian

import pickle as plk
from scipy.sparse import csr_matrix
gpu_options = tf.GPUOptions(allow_growth=True)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
keras.backend.tensorflow_backend.set_session(sess)

Using TensorFlow backend.


In [3]:
data_facial_tra = plk.load(open('../data_clean/split/data_visual_tra', 'rb'))
data_facial_pre_tra = plk.load(open('../data_clean/split/data_visual_pre_tra', 'rb'))

data_facial_tes = plk.load(open('../data_clean/split/data_visual_tes', 'rb'))
data_facial_pre_tes = plk.load(open('../data_clean/split/data_visual_pre_tes', 'rb'))


# data_utt_ori_tra = plk.load(open('../data_clean/split/data_utt_ori_tra', 'rb'))
data_utt_EA_tra = plk.load(open('../data_clean/split/data_utt_EA_tra', 'rb'))
# data_utt_ori_pre_tra = plk.load(open('../data_clean/split/data_utt_ori_pre_tra', 'rb'))
data_utt_EA_pre_tra = plk.load(open('../data_clean/split/data_utt_EA_pre_tra', 'rb'))

data_utt_pre_tes = plk.load(open('../data_clean/split/data_utt_pre_tes', 'rb'))
data_utt_tes = plk.load(open('../data_clean/split/data_utt_tes', 'rb'))



data_txt_tra = plk.load(open('../data_clean/split/data_txt_tra', 'rb'))
data_txt_pre_tra = plk.load(open('../data_clean/split/data_txt_pre_tra', 'rb'))

data_txt_tes = plk.load(open('../data_clean/split/data_txt_tes', 'rb'))
data_txt_pre_tes = plk.load(open('../data_clean/split/data_txt_pre_tes', 'rb'))


data_gens_tra = plk.load(open('../data_clean/split/data_gens_tra', 'rb'))
data_gens_tes = plk.load(open('../data_clean/split/data_gens_tes_', 'rb'))


data_emos_tra = plk.load(open('../data_clean/split/data_emos_tra_', 'rb'))

data_emos_tes = plk.load(open('../data_clean/split/data_emos_tra_', 'rb'))

In [ ]:
data_facial_tra = np.reshape(data_facial_tra, (5810, 512,1))
data_facial_pre_tra = np.reshape(data_facial_pre_tra, (5810, 512,1))

data_facial_tes = np.reshape(data_facial_tes, (1623, 512,1))
data_facial_pre_tes = np.reshape(data_facial_pre_tes, (1623, 512,1))


data_utt_EA_tra = np.asarray(data_utt_EA_tra)
data_utt_EA_pre_tra = np.asarray(data_utt_EA_pre_tra)

data_utt_pre_tes = np.asarray(data_utt_pre_tes)
data_utt_tes = np.asarray(data_utt_tes)


data_txt_tra = np.asarray(data_txt_tra)
data_txt_pre_tra = np.asarray(data_txt_pre_tra)

data_txt_tes = np.asarray(data_txt_tes)
data_txt_pre_tes = np.asarray(data_txt_pre_tes)


data_gens_tra = to_categorical(data_gens_tra, 2)
data_gens_tes = to_categorical(data_gens_tes, 2)

data_emos_tra = to_categorical(data_emos_tra, 6)
data_emos_tes = to_categorical(data_emos_tes, 6)

In [ ]:
print('data_facial_tra', data_facial_tra.shape)
print('data_facial_pre_tra', data_facial_pre_tra.shape)

print('data_facial_tes', data_facial_tes.shape)
print('data_facial_pre_tes', data_facial_pre_tes.shape)

print('data_utt_EA_tra', np.shape(data_utt_EA_tra))
print('data_utt_EA_pre_tra', np.shape(data_utt_EA_pre_tra))

print('data_utt_pre_tes', np.shape(data_utt_pre_tes))
print('data_utt_tes', np.shape(data_utt_tes))

print('data_txt_tra', np.shape(data_txt_tra))
print('data_txt_pre_tra', np.shape(data_txt_pre_tra))

print('data_txt_tes', np.shape(data_txt_tes))
print('data_txt_pre_tes', np.shape(data_txt_pre_tes))

print('data_gens_tra', np.shape(data_gens_tra))
print('data_gens_tes', np.shape(data_gens_tes))

print('data_emos_tra', np.shape(data_emos_tra))
print('data_emos_tes', np.shape(data_emos_tes))

print('data_emos_next_tra', np.shape(data_emos_next_tra))
print('data_emos_next_tes', np.shape(data_emos_next_tes))

In [ ]:
Trainable = True

facial_model = load_model('./facial/facial_model.h5', 
                          custom_objects={'MultiHeadAttention':MultiHeadAttention, 'EmoEncDec': EmoEncDec})

for layer in facial_model.layers:
    layer.name = layer.name + str("_facial")
    layer.trainable = Trainabl
    
audio_model = load_model('./audio/audio_model.h5', 
                          custom_objects={'MultiHeadAttention':MultiHeadAttention, 'EmoEncDec': EmoEncDec})

for layer in audio_model.layers:
    layer.name = layer.name + str("_audio")
    layer.trainable = Trainable
    
text_model = load_model('./text/text_model.h5', 
                          custom_objects={'MultiHeadAttention':MultiHeadAttention})

for layer in text_model.layers:
    layer.name = layer.name + str("_text")
    layer.trainable = Trainable

facial_model.summary()
audio_model.summary()
text_model.summary()

In [ ]:
classes = 6
batch = 16
epochs = 5000
l2_reg = 5e-4 

In [ ]:
classes = 6
batch = 16
epochs = 5000
l2_reg = 5e-4 

In [ ]:
A_inter_in = Input(tensor=A_inter, sparse=False)


facial_hid = facial_model.layers[-2].output
audio_hid = audio_model.layers[-3].output
text_hid = text_model.layers[-3].output

text_hid = Dense(64)(text_hid)

merge = Concatenate(axis=-1)([facial_hid, audio_hid, text_hid])

merge_inter= Reshape((3,64))(merge)

gat_inter_inter  = GraphAttention(32,
                                   attn_heads=8,
                                   concat_heads=True,
                                   dropout_rate=0.3,
                                   activation='elu',
                                   kernel_regularizer=l2(l2_reg),
                                   attn_kernel_regularizer=l2(l2_reg)
                                   )([merge_inter, A_inter_in])
R_gra = Flatten()(gat_inter_inter)

R = Dense(64)(R_gra)
emo = Dense(6, name='emo', activation='softmax')(R)
gen = Dense(2, name='gen', activation='softmax')(audio_hid)

# R_next = Dense(64)(R_gra)
# emo_next = Dense(6, name='emo_next', activation='softmax')(R_next)

model = Model(inputs=[facial_model.inputs[0], facial_model.inputs[1],
                      audio_model.inputs[0], audio_model.inputs[1],
                      text_model.inputs[0], text_model.inputs[1],
                      A_inter_in
                     ],
               outputs=[emo, gen])
model.summary()

In [ ]:
file_path_root = './combined/'
model_file = file_path_root+'Text_AudioEA_Faceori.h5'
callback_list = [
                    TensorBoard(log_dir=file_path_root),
                    EarlyStopping(
                        monitor='val_emo_acc',
                        patience=100,
                        verbose=1,
                        mode='auto'
                    ),
                    ModelCheckpoint(
                        filepath=model_file,
                        monitor='val_emo_acc',
                        save_best_only='True',
                        verbose=1,
                        mode='auto',
                        period=1
                    )
                    ]

In [ ]:
model.compile(optimizer='adam', 
              loss={'emo':'categorical_crossentropy',
                    'gen':'categorical_crossentropy',
                    },
              loss_weights={'emo':1.,
                            'gen':1.,
                            },
              metrics=['acc'])

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd

samples = 1623


def batch_generator(X1, X2, X3, X4, X5, X6, Y, batch_size = 3):
    indices = np.arange(samples) 
    batch=[]
    while True:
            for i in indices:
                batch.append(i)
                if len(batch)==batch_size:
                    yield ([X1[batch], X2[batch], X3[batch], X4[batch], X5[batch], X6[batch], A_inter], Y[batch])
                    batch=[]

model = load_model(model_file, custom_objects={'MultiHeadAttention':MultiHeadAttention, 'EmoEncDec': EmoEncDec, 'GraphAttention':GraphAttention})
predicted_test_labels = model.predict_generator(batch_generator(data_facial_pre_tes, data_facial_tes,
                            data_utt_pre_tes, data_utt_tes,
                            data_txt_pre_tes, data_txt_tes, data_emos_tes), steps = 541)[0].argmax(axis=1)

numeric_test_labels = np.array(data_emos_tes).argmax(axis=1)

report_filename = file_path_root+'Results.txt' 

with open(report_filename, 'w', encoding='utf-8') as f:
    print(classification_report(numeric_test_labels, predicted_test_labels_t, target_names = ['hap', 'sad', 'neu', 'ang', 'exc', 'fru']), file=f)
print(classification_report(numeric_test_labels, predicted_test_labels_t, target_names = ['hap', 'sad', 'neu', 'ang', 'exc', 'fru']))
labels = ['hap', 'sad', 'neu', 'ang', 'exc', 'fru']
print('   '+' '.join(labels))
cm = confusion_matrix(y_true=numeric_test_labels.tolist(), y_pred=predicted_test_labels_t.tolist())
print(cm)


nor_cm = []
for i in range(6):
    row_sum = cm[i].sum()
#     print(row_sum)
    l_n = []
    for j in range(6):
#         confusion_mat[i][j] = confusion_mat[i][j]/row_sum
        l_n.append(cm[i][j]/row_sum)
#         print(confusion_mat[i][j]/row_sum)
    nor_cm.append(l_n)
    
df_cm = pd.DataFrame(nor_cm, index = [i for i in ['hap', 'sad', 'neu', 'ang', 'exc', 'fru']],
                  columns = [i for i in ['hap', 'sad', 'neu', 'ang', 'exc', 'fru']])

sn.heatmap(df_cm,  annot=True)
plt.savefig(file_path_root+'cm.png')

cm = np.transpose(cm)

nor_cm = []
for i in range(6):
    row_sum = cm[i].sum()
#     print(row_sum)
    l_n = []
    for j in range(6):
#         confusion_mat[i][j] = confusion_mat[i][j]/row_sum
        l_n.append(cm[i][j]/row_sum)
#         print(confusion_mat[i][j]/row_sum)
    nor_cm.append(l_n)
    
df_cm = pd.DataFrame(nor_cm, index = [i for i in ['hap', 'sad', 'neu', 'ang', 'exc', 'fru']],
                  columns = [i for i in ['hap', 'sad', 'neu', 'ang', 'exc', 'fru']])

sn.heatmap(df_cm,  annot=True)
plt.savefig(file_path_root+'cm_precision.png')